<a href="https://colab.research.google.com/github/Jas7777/AHS-Hackathon/blob/main/Facial_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Run this cell to load some data! 
!pip -q install streamlit > /dev/null
!pip -q install pyngrok > /dev/null
!wget https://www.dropbox.com/s/072b5vf4b33bu1l/emotion_detection_model_for_streamlit.h5 > /dev/null
!wget https://www.dropbox.com/s/p52z1qle0x1uf6f/happy.png > /dev/null

!pip uninstall tornado -y
!yes | pip install tornado==5.1.0

import tensorflow as tf
import pandas as pd
import cv2
import plotly.express as px
import numpy as np
from pyngrok import ngrok

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.6.6 requires traitlets>=5.2.2, but you have traitlets 5.1.1 which is incompatible.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.30 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.15.1 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.34.0 which is incompatible.
google-colab 1.0.0 requires tornado~=5.1.0; python_version >= "3.0", but you have tornado 6.2 which is incompatible.
--2022-07-11 02:56:39--  https://www.dropbox.com/s/072b5vf4b33bu1l/emotion_detection_model_for_streamlit.h5
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaitin

In [ ]:
!ngrok authtoken 29nmRwsadmEHrVqrXOmdWnnkibn_4b7tAb7mLGwVwYwVQc2zX # paste your authtoken here

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
#@title Run this cell to initialize your website!
%%writefile app.py
import streamlit as st

st.title("Face Teller")
st.header("Tells the emotion")

Writing app.py


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import cv2
import plotly.express as px
import tensorflow as tf

model = tf.keras.models.load_model("emotion_detection_model_for_streamlit.h5")
model.summary()
model.input_shape
st.title("Face Teller")
st.header("Tells the emotion")
# ====== Save model ========
model.save("emotion_detection_model_for_streamlit.h5")

# ====== Load model ========
tf.keras.models.load_model("emotion_detection_model_for_streamlit.h5")


f = st.file_uploader("Upload Image")
if f is not None: 
  file_bytes = np.asarray(bytearray(f.read()), dtype=np.uint8)
  image = cv2.imdecode(file_bytes, 1)
  print("Shape of image:")
  print(image.shape)
  print(f"Maximum value in the image: ")
  print(image.max())
  print("Minimum value in the image:")
  print(image.min())
  st.write(px.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)))
  resized = cv2.resize(image, (48, 48), interpolation=cv2.INTER_LANCZOS4)
  print(resized.shape)


  gray_1d = np.mean(resized, axis=-1)
  gray = np.zeros_like(resized)
  
  gray[:,:,0] = gray_1d
  gray[:,:,1] = gray_1d
  gray[:,:,2] = gray_1d
  px.imshow(gray)
  normalized = gray/255
  st.write(px.imshow(normalized))
  print("Shape of image:")
  print(normalized.shape)
  print(f"Maximum value in the image: ")
  print(normalized.max())
  print("Minimum value in the image:")
  print(normalized.min())
  EMOTIONS = ['ANGRY', 'HAPPY', 'SAD', 'SURPRISE', 'NEUTRAL']



  model_input = np.expand_dims(normalized,0)
  scores = model.predict(model_input).flatten()
  scores
  
  df = pd.DataFrame()
  df["Emotion"] = EMOTIONS
  df["Scores"] = scores
  st.write(px.bar(df, x='Emotion', y='Scores', title="Model scores for each emotion"))


  prediction = EMOTIONS[scores.argmax()] # Fill this in!
  print(prediction)

Overwriting app.py


In [ ]:
from pyngrok import ngrok
#Publish Web App (Run this again whenever you make changes)
public_url = ngrok.connect(port='80')
print (public_url)
! streamlit run --server.port 80 app.py

NgrokTunnel: "http://263d-35-229-71-160.ngrok.io" -> "http://localhost:80"
2022-07-11 02:56:59.996 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:80
  External URL: http://35.229.71.160:80

2022-07-11 02:57:13.887498: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_23 (Dense)            (None, 1024)              525312    